In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

In [ ]:
using BoSSS.Foundation.Quadrature;
using BoSSS.Foundation.XDG;
using MathNet.Numerics;
using MathNet.Numerics.Interpolation;
using MathNet.Numerics.RootFinding;

In [ ]:
// var myDb = OpenOrCreateDatabase(@"C:\Databases\HeatedCavity_RayleighSweepStudy");
var myDb = OpenOrCreateDatabase(@"C:\Databases\HeatedBackwardFacingStep_longCalc2");

In [ ]:
myDb.Sessions[0].Timesteps[0].Fields.ForEach(F=>Console.WriteLine(F.Identification))

In [ ]:
myDb.Sessions[0].Timesteps[0].Fields.Where(F=>F.Identification == "Temperature").SingleOrDefault()

Temperature

In [ ]:
static public CubicSpline SplineOnEdge(EdgeMask em, DGField field, out double lower_Bound, out double upper_Bound, double offset = 0.0, int d = 0){

    var grd = field.GridDat;

    List<double> nodes = new List<double>();
    List<double> values = new List<double>();

    EdgeQuadrature.GetQuadrature(new int[] { 1 }, grd,
        (new EdgeQuadratureScheme(true, em)).Compile(grd, field.Basis.Degree * 2),
        delegate (int i0, int Length, QuadRule QR, MultidimensionalArray EvalResult) {        

            MultidimensionalArray DummyIN = MultidimensionalArray.Create(Length, QR.NoOfNodes);
            MultidimensionalArray DummyOT = MultidimensionalArray.Create(Length, QR.NoOfNodes);        

            MultidimensionalArray GlobalNodes = MultidimensionalArray.Create(Length, QR.NoOfNodes, 2);        
            
            if(field is XDGField xField){
                xField.GetSpeciesShadowField("A").EvaluateEdge(i0, Length, QR.Nodes, DummyIN, DummyOT, null, null, null, null, 0, 0.0); 
            } else{
                field.EvaluateEdge(i0, Length, QR.Nodes, DummyIN, DummyOT, null, null, null, null, 0, 0.0); 
            }  

            for(int i = 0; i < Length; i++){  
                int iTrafo = ((GridData)grd).Edges.Edge2CellTrafoIndex[i0+i, 0];
                NodeSet volNodeSet = QR.Nodes.GetVolumeNodeSet(grd, iTrafo);
                int jCell = ((GridData)grd).Edges.CellIndices[i0+i, 0];
                grd.TransformLocal2Global(volNodeSet, jCell, 1, GlobalNodes, i);          
                int K = QR.NoOfNodes;
                for(int k = 0; k < K; k++){
                    nodes.Add(GlobalNodes[i, k, d]);
                    values.Add(DummyIN[i, k] - offset);
                }
            }

        },delegate (int i0, int Length, MultidimensionalArray ResultsOfIntegration) {
        }).Execute(); 

    lower_Bound = nodes.Min();
    upper_Bound = nodes.Max();

    return CubicSpline.InterpolateAkima(nodes.ToArray(), values.ToArray());
}

In [ ]:
// Finds the position where the specified field assumes the given value
static public double PosOfValueOnEdge(double value, EdgeMask em, DGField field, int d = 0){    
    
    var Spline = SplineOnEdge(em, field, out double lB, out double uB, value, d);
    double root = Bisection.FindRoot(t => Spline.Interpolate(t), lB, uB);    
    
    return root;
}

// Finds the position where the specified field assumes an extremum, in the given direction
static public double PosOfExtremumOnEdge(EdgeMask em, DGField field, int d){    

    var dField = field.CloneAs();
    dField.Clear();
    dField.Derivative(1.0, field, d);
    
    double root = PosOfValueOnEdge(0.0, em, dField, d);
    
    return root;
}

Calculation of Local Nusselt Number $\text{Nu} := \frac{S}{T0-T1}\vec{n}\cdot \nabla T$


In [ ]:

static public double[] LocalNusseltNumber(ISessionInfo si, double[] coordinates, double deltaT, int dirGradient, int dirSpline ) {
    DGField Temperature       = si.Timesteps.Last().Fields.Where(f => f.Identification == "Temperature").SingleOrDefault();
    var grd = (GridData)Temperature.GridDat;
    Console.WriteLine("1");
    var GradT_d = (DGField)Temperature.Clone();
    GradT_d.Clear();
    GradT_d.Derivative(1.0, Temperature, dirGradient);

    if(dirGradient == dirSpline){
    throw new Exception();
}
    EdgeMask em = new EdgeMask(grd, X => X[1] + 0.0 < 1e-12); //
    Console.WriteLine("2");
    var spline = SplineOnEdge(em, GradT_d, out double _a, out double  _b, d:dirSpline);
    Console.WriteLine("3");
    
    double[] Nusselt = new double[coordinates.Length];
    for(int i = 0; i < coordinates.Length; i++){
        double GradT = spline.Interpolate(coordinates[i]);    
        Nusselt[i] = GradT/deltaT;
    }   
    Console.WriteLine("4");
    return Nusselt;
}

In [ ]:
var sess = myDb.Sessions[0];
double[] x = GenericBlas.Linspace(0,50, 500);
var res = LocalNusseltNumber(sess, x, 1.2, 1, 0 );



1
2
3
4


In [ ]:
for(int i = 0;i < res.Length;i++){
// res[i] = res[i]*-1;
res[i] = res[i]*-1;

}

In [ ]:
LineColors[]  allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle,PointTypes.OpenLowerTriangle, };
var plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;
fmt.PointType =  PointTypes.Diamond;
fmt.LineColor = LineColors.Blue;
plot.AddDataGroup( "Nu_h",x, res, fmt);

In [ ]:
var gp = plot.ToGnuplot();
gp.PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 1.6 
 
 
 
 
 1.8 
 
 
 
 
 2 
 
 
 
 
 0 
 
 
 
 
 10 
 
 
 
 
 20 
 
 
 
 
 30 
 
 
 
 
 40 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Nuh 
 
 
 Nu h 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M655.1,57.1 L708.5,57.1 M53.9,386.3 L55.3,416.0 L56.7,421.7 L58.1,422.5 L59.5,421.8 L60.9,421.4
 L62.3,422.1 L63.7,424.6 L65.1,425.2 L66.5,423.9 L67.9,423.9 L69.2,426.4 L70.6,431.3 L72.0,436.2
 L73.4,438.5 L74.8,437.1 L76.2,433.9 L77.6,430.1 L79.0,426.1 L80.4,423.0 L81.8,421.9 L83.2,422.0
 L84.6,420.4 L86.0,418.4 L87.4,417.3 L88.8,416.7 L90.2,416.0 L91.6,414.0 L93.0,411.3 L94.4,409.2
 L95.8,407.2 L97.2,405.3 L98.5,403.2 L99.9,398.7 L101.3,396.2 L102.7,410.1 L104.1,410.4 L105.5,409.7
 L106.9,408.9 L108.3,407.9 L109.7,406.7 L111.1,405.5 L112.5,404.5 L113.9,403.8 L115.3,403.3 L116.7,403.1
 L118.1,402.6 L119.5,401.6 L120.9,400.6 L122.3,400.1 L123.7,399.7 L125.1,399.4 L126.4,398.8 L127.8,398.0
 L129.2,397.0 L130.6,395.7 L132.0,394.6 L133.4,394.2 L134.8,394.3 L136.2,393.8 L137.6,392.6 L139.0,391.5
 L140.4,390.8 L141.8,390.2 L143.2,389.6 L144.6,388.8 L146.0,387.9 L147.4,386.9 L148.8,385.7 L150.2,384.7
 L151.6,383.8 L153.0,382.9 L154.4,382.2 L155.7,382.0 L157.1,381.8 L158.5,381.3 L159.9,380.5 L161.3,379.3
 L162.7,378.0 L164.1,376.2 L165.5,374.3 L166.9,372.4 L168.3,370.7 L169.7,369.2 L171.1,368.2 L172.5,367.4
 L173.9,366.7 L175.3,365.9 L176.7,365.2 L178.1,364.5 L179.5,363.8 L180.9,363.4 L182.3,362.6 L183.7,361.9
 L185.0,360.8 L186.4,359.8 L187.8,358.8 L189.2,358.0 L190.6,357.2 L192.0,356.5 L193.4,355.8 L194.8,355.1
 L196.2,354.3 L197.6,353.4 L199.0,352.6 L200.4,351.9 L201.8,351.3 L203.2,350.5 L204.6,349.7 L206.0,348.9
 L207.4,348.1 L208.8,347.3 L210.2,346.6 L211.6,345.8 L213.0,345.1 L214.3,344.2 L215.7,343.3 L217.1,342.3
 L218.5,341.9 L219.9,341.2 L221.3,340.4 L222.7,339.5 L224.1,338.6 L225.5,337.8 L226.9,336.9 L228.3,336.1
 L229.7,335.3 L231.1,334.4 L232.5,333.6 L233.9,332.7 L235.3,331.7 L236.7,330.8 L238.1,329.9 L239.5,329.0
 L240.9,328.1 L242.3,327.2 L243.6,326.3 L245.0,325.5 L246.4,324.6 L247.8,323.9 L249.2,323.1 L250.6,322.4
 L252.0,321.7 L253.4,321.0 L254.8,320.3 L256.2,319.7 L257.6,319.1 L259.0,318.1 L260.4,316.8 L261.8,315.9
 L263.2,315.0 L264.6,314.1 L266.0,313.0 L267.4,311.9 L268.8,310.8 L270.2,309.8 L271.5,308.9 L272.9,307.9
 L274.3,306.9 L275.7,305.8 L277.1,304.8 L278.5,304.0 L279.9,303.2 L281.3,302.4 L282.7,301.7 L284.1,300.9
 L285.5,300.0 L286.9,298.9 L288.3,297.7 L289.7,296.3 L291.1,295.3 L292.5,294.6 L293.9,293.8 L295.3,292.7
 L296.7,291.8 L298.1,290.9 L299.5,289.9 L300.8,289.1 L302.2,288.6 L303.6,288.2 L305.0,287.1 L306.4,286.0
 L307.8,285.3 L309.2,285.3 L310.6,287.9 L312.0,284.8 L313.4,280.4 L314.8,279.2 L316.2,278.2 L317.6,277.4
 L319.0,276.8 L320.4,275.9 L321.8,275.0 L323.2,274.2 L324.6,273.8 L326.0,273.0 L327.4,269.9 L328.8,269.9
 L330.1,270.0 L331.5,269.3 L332.9,268.5 L334.3,267.3 L335.7,266.3 L337.1,265.7 L338.5,265.0 L339.9,264.1
 L341.3,262.8 L342.7,261.4 L344.1,260.1 L345.5,258.8 L346.9,257.3 L348.3,256.1 L349.7,254.9 L351.1,253.6
 L352.5,252.3 L353.9,251.0 L355.3,249.6 L356.7,248.5 L358.1,247.5 L359.4,246.6 L360.8,245.5 L362.2,244.8
 L363.6,244.1 L365.0,243.3 L366.4,241.3 L367.8,240.0 L369.2,238.8 L370.6,237.5 L372.0,236.1 L373.4,234.5
 L374.8,232.9 L376.2,231.4 L377.6,230.1 L379.0,228.8 L380.4,227.6 L381.8,226.4 L383.2,225.2 L384.6,223.9
 L386.0,222.5 L387.4,221.1 L388.7,219.5 L390.1,218.0 L391.5,216.4 L392.9,214.8 L394.3,213.3 L395.7,211.7
 L397.1,210.1 L398.5,208.7 L399.9,207.3 L401.3,205.9 L402.7,204.6 L40

In [ ]:
var sess = myDb.Sessions[0];


In [ ]:
myDb.Sessions[0].Export().Do()

Starting export process... Data will be written to the directory: C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\HeatedBackwardFacingStep_longCalc2__Heated_BackwardFacingStep_Lichtenberg_2ER25lLev_96Cores_16Nodesdg2k16Re1000EnergyOK1__aa3857a6-6eae-4880-a951-0008cd19043e


C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\HeatedBackwardFacingStep_longCalc2__Heated_BackwardFacingStep_Lichtenberg_2ER25lLev_96Cores_16Nodesdg2k16Re1000EnergyOK1__aa3857a6-6eae-4880-a951-0008cd19043e

In [ ]:
DGField Temperature       = sess.Timesteps.Last().Fields.Single(f => f.Identification == "Temperature");
// var grd = (GridData)Temperature.GridDat;

Error: System.OperationCanceledException: Command :SubmitCode: DGField Temperature       = sess.Timesteps.Last(). ... cancelled.

In [ ]:
double tin = 10 +273;
double thot = 40+273;
double tref = tin;

double TIN = tin/tref;
double THOT = thot/tref;

In [ ]:
for(int i = 0; i< res.Length; i++){
    res[i] = res[i]*1.2/(30.0/273);
}

0.10989010989010989